# BOOK RECOMMENDER SYSTEM

This project is made under the lab work of Software Engineering Lab (CSD-327).<br/> 
In this project we have used the Collaborative Filtering Algorithm for recommending the top 10 books to the particular user depending upon the books , which that particular user had already reviewed.

## To publish this project on web, we have used the Anvil Works. 

### So, firstly we will install the Anvil Uplink Library.

In [ ]:
pip install anvil-uplink

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp37-none-any.whl size=45216 sha256=6917db53e91c4b72018d60f7470af03a0b6db73393bc9346aa137e67f79e154a
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


### We will connect our Google Colaboratory with the Anvil Server by using the Uplink Key of our web application.

In [ ]:
import anvil.server
anvil.server.connect("IKCBCGKEMUGTYT7PKD35GDWS-FXX7AGOTRWYZASHR")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


## Installing necessary python libraries.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

### Changing directory.

In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


## Pre-processing of Data.

Task is to make our data set ready to feed directly to the model.

### Reading the .csv data files as dataframes.

**Data Source** : http://www2.informatik.uni-freiburg.de/~cziegler/BX/ <br/>
**BX-Book-Ratings.csv** : This file contain data, that how user have rated different books with given ISBN number.<br/>
**BX-Users.csv** : This file contain data, about different User Ids of users.<br/>
**Data/BX-Books.csv** : This file contain data, about information of books according to ISBN number.

In [ ]:
rating = pd.read_csv('Data/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('Data/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
book = pd.read_csv('Data/BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:27

### Merge book and rating data.

We will merge book and rating data to get the rating of all books with the all the information of the given books. We will then clean the data by dropping the unnecessary data. Then we will, choose only the books, which will have more than 25 reviews.

#### Merging book and rating data.

In [ ]:
book_rating = pd.merge(rating, book, on='ISBN')
book_rating.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


#### Dropping unnecessary data.

In [ ]:
cols = ['Year-Of-Publication', 'Publisher', 'Book-Author', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
book_rating.drop(cols, axis=1, inplace=True)
book_rating.head()

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel


#### We will count, which book title has got how many reviews.

In [ ]:
rating_count = (book_rating.
     groupby(by = ['Book-Title'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'RatingCount_book'})
     [['Book-Title', 'RatingCount_book']]
)
rating_count.head()

,Book-Title,RatingCount_book
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


#### Choosing books which are rerviwed more than 25 times. This will help to get bettter result from the model.

In [ ]:
threshold = 25
rating_count = rating_count.query('RatingCount_book >= @threshold')
rating_count.head()

,Book-Title,RatingCount_book
75,'Salem's Lot,47
203,10 Lb. Penalty,61
422,101 Dalmatians,37
673,"14,000 Things to Be Happy About",28
697,16 Lighthouse Road,65


### Merging user data with how they have reviewed the books.

We will merge user data with how they have reviewed the books to get the rating of all books with respective users. Then we will, choose the data, in which user has rated at least 20 books.

#### Merging data.

In [ ]:
user_rating = pd.merge(rating_count, book_rating, left_on='Book-Title', right_on='Book-Title', how='left')
user_rating.head()

,Book-Title,RatingCount_book,User-ID,ISBN,Book-Rating
0,'Salem's Lot,47,8936,067103975X,0
1,'Salem's Lot,47,172245,067103975X,0
2,'Salem's Lot,47,189835,067103975X,5
3,'Salem's Lot,47,9226,0451168089,0
4,'Salem's Lot,47,33283,0451168089,10


#### We will count how many time a user has reviewed the books.

In [ ]:
user_count = (user_rating.
     groupby(by = ['User-ID'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'RatingCount_user'})
     [['User-ID', 'RatingCount_user']]
)
user_count.head()

,User-ID,RatingCount_user
0,8,2
1,9,2
2,10,1
3,14,1
4,16,2


#### Choosing users who have reviewed more than 20 books.

In [ ]:
threshold = 20
user_count = user_count.query('RatingCount_user >= @threshold')
user_count.head()

,User-ID,RatingCount_user
52,243,68
54,254,139
89,487,21
96,507,61
115,638,51


### Combining the cleaned data to get the final data ready for the model.

In [ ]:
combined = user_rating.merge(user_count, left_on = 'User-ID', right_on = 'User-ID', how = 'inner')
print('Number of unique books: ', combined['Book-Title'].nunique())
print('Number of unique users: ', combined['User-ID'].nunique())
combined.head()

Number of unique books:  5850
Number of unique users:  3192


,Book-Title,RatingCount_book,User-ID,ISBN,Book-Rating,RatingCount_user
0,'Salem's Lot,47,8936,067103975X,0,177
1,1st to Die: A Novel,509,8936,0446610038,0,177
2,A Case of Need,236,8936,0451210638,0,177
3,A Perfect Stranger,54,8936,0440168724,0,177
4,Accident,126,8936,0440217547,0,177


#### We will save a csv file of the final data, if we need it further.

In [ ]:
combined.to_csv(r'Data/combined.csv', index = False)

#### We will normalize the rating feature, using MinMax scaler.

In [ ]:
scaler = MinMaxScaler()
combined['Book-Rating'] = combined['Book-Rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['Book-Rating'].values.reshape(-1,1)))
combined['Book-Rating'] = rating_scaled
rating_scaled.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


#### We will drop the all duplicate values and will save all the data in a numpy array for use in model.

In [ ]:
combined = combined.drop_duplicates(['User-ID', 'Book-Title'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-Title', values='Book-Rating')
user_book_matrix.fillna(0, inplace=True)
users = user_book_matrix.index.tolist()
books = user_book_matrix.columns.tolist()
user_book_matrix = user_book_matrix.to_numpy()

## Creating the model.

### tf.placeholder is available in v1 so we will disable v2.

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


### Choose dimensions of hidden layers and initialize weight and bias vectors.

In [ ]:
num_input = combined['Book-Title'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

### Defining the encoder and decoder model.

In [ ]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

### Constructing the model

In [ ]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

### Define Loss Function (Mean Squared) and Optimizer(RMS Prop) to minimize the squared error, and define the Evaluation Metrics.

In [ ]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Initialize the variables, then create an empty data frame to store the result table,

In [ ]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

### Training the model.

Split training data into batches, and feed the network with them. <br/>
Store outpu in data frame initialised above.

In [ ]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35
    session.run(init)
    session.run(local_init)
    num_batches = int(user_book_matrix.shape[0] / batch_size)
    user_book_matrix = np.array_split(user_book_matrix, num_batches)  
    for i in range(epochs):
        avg_cost = 0
        for batch in user_book_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l
        avg_cost /= num_batches
        print("epoch: {} Loss: {}".format(i + 1, avg_cost))
    user_book_matrix = np.concatenate(user_book_matrix, axis=0)
    preds = session.run(decoder_op, feed_dict={X: user_book_matrix})
    pred_data = pred_data.append(pd.DataFrame(preds))

epoch: 1 Loss: 0.3639434225611634
epoch: 2 Loss: 0.29699674739942444
epoch: 3 Loss: 0.06867655212121022
epoch: 4 Loss: 0.004275028215168597
epoch: 5 Loss: 0.0036841153991050445
epoch: 6 Loss: 0.003619934022835978
epoch: 7 Loss: 0.003564233858404415
epoch: 8 Loss: 0.003409412610191074
epoch: 9 Loss: 0.0033989241372302664
epoch: 10 Loss: 0.0033915268953193674
epoch: 11 Loss: 0.0033462174405640625
epoch: 12 Loss: 0.003130413851557443
epoch: 13 Loss: 0.003039404161397245
epoch: 14 Loss: 0.003028823153607056
epoch: 15 Loss: 0.00286489057451855
epoch: 16 Loss: 0.0028619294599783454
epoch: 17 Loss: 0.0028602686649249805
epoch: 18 Loss: 0.0028587833525367326
epoch: 19 Loss: 0.0028493811125652148
epoch: 20 Loss: 0.0026901640972777545
epoch: 21 Loss: 0.0026876374536736326
epoch: 22 Loss: 0.002686761807046503
epoch: 23 Loss: 0.0026860240562870115
epoch: 24 Loss: 0.0026853690985692564
epoch: 25 Loss: 0.002684779925742647
epoch: 26 Loss: 0.0026842461115997897
epoch: 27 Loss: 0.002683760055883245
ep

### Must remove the books from the data, which user has already reviewed.

In [ ]:
pred_data = pred_data.stack().reset_index(name='Book-Rating')
pred_data.columns = ['User-ID', 'Book-Title', 'Book-Rating']
pred_data['User-ID'] = pred_data['User-ID'].map(lambda value: users[value])
pred_data['Book-Title'] = pred_data['Book-Title'].map(lambda value: books[value])

### Define the function to get the Top 10 books for particular user.

In [ ]:
keys = ['User-ID', 'Book-Title']
index_1 = pred_data.set_index(keys).index
index_2 = combined.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['User-ID', 'Book-Rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('User-ID').head(10)

## Preparing functions for Anvil Server call, to be used in web application. 

### For getting Top 10 reommendations.

In [ ]:
@anvil.server.callable
def result1(x) : 
  z = top_ten_ranked.loc[top_ten_ranked['User-ID'] == x]
  a = z.drop(["User-ID","Book-Rating"],axis =1)
  b = a.to_numpy()
  return b

### For getting insight, user has reviewed which of the books.

In [ ]:
@anvil.server.callable
def result2(x) :
  p = book_rating.loc[book_rating['User-ID'] == x].sort_values(by=['Book-Rating'], ascending=False)
  q = p.drop(["User-ID","ISBN"],axis =1)
  r = q.to_numpy()
  return r

### Used to constanly use Google Colab for Web Application.

In [ ]:
anvil.server.wait_forever()

## To get insight what is output of functions we are using.

### Top 10 recommendations.

In [ ]:
top_ten_ranked.loc[top_ten_ranked['User-ID'] == 243]

,User-ID,Book-Title,Book-Rating
4302,243,The Da Vinci Code,0.061805
5060,243,The Secret Life of Bees,0.047085
1837,243,Harry Potter and the Chamber of Secrets (Book 2),0.045396
1841,243,Harry Potter and the Prisoner of Azkaban (Book 3),0.043794
702,243,Bridget Jones's Diary,0.042827
2406,243,Life of Pi,0.041292
1843,243,Harry Potter and the Sorcerer's Stone (Harry P...,0.038947
4984,243,The Red Tent (Bestselling Backlist),0.038819
5401,243,To Kill a Mockingbird,0.037897
1839,243,Harry Potter and the Goblet of Fire (Book 4),0.037194


### Books reviewed by user.

In [ ]:
book_rating.loc[book_rating['User-ID'] == 243].sort_values(by=['Book-Rating'], ascending=False)

,User-ID,ISBN,Book-Rating,Book-Title
96475,243,0375400117,10,Memoirs of a Geisha
90540,243,0060915544,10,The Bean Trees
150644,243,0446364800,9,The General's Daughter
189970,243,0425163407,9,Unnatural Exposure
48114,243,0316776963,9,Me Talk Pretty One Day
...,...,...,...,...
188830,243,0312169787,0,The Red Tent : A Novel
188859,243,0312872682,0,The James Dean Affair: A Neil Gulliver &amp; S...
188865,243,0316602906,0,The Big Bad Wolf: A Novel
188985,243,0316693006,0,Four Blind Mice
